In [2]:
%tensorflow_version 1.x
from tensorflow.contrib.layers import batch_norm, flatten
from keras.models import model_from_json
import os
import openpyxl
import sys
import tensorflow as tf
import numpy as np
import math
from keras.callbacks import EarlyStopping
import random
from random import seed
from keras import backend as K
import gc
seed(1)
#os.environ["CUDA_VISIBLE_DEVICES"] = str(2)  # gpu_id
from google.colab import drive
drive.mount('/content/gdrive')

TensorFlow 1.x selected.


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
from tensorflow.python.client import device_lib

device_name = tf.test.gpu_device_name()
print('Found GPU at: {}'.format(device_name))
file_path = '/content/gdrive/My Drive/Colab Notebooks/k-fold/'


Found GPU at: 


In [0]:
def create_model(models,keep_prob,layer_num):
  #tf.nn.relu
  #tf.nn.leaky_relu
  if layer_num ==3:
    model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(models[0]),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(tf.nn.relu),
      tf.keras.layers.Dropout(rate=1-keep_prob), 

      tf.keras.layers.Dense(models[1]),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(tf.nn.relu),
      tf.keras.layers.Dropout(rate=1-keep_prob),    

      tf.keras.layers.Dense(models[2]),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(tf.nn.relu),
      tf.keras.layers.Dropout(rate=1-keep_prob), 

      tf.keras.layers.Dense(2, activation=tf.nn.softmax)
    ])

  elif layer_num == 4:
      model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(models[0]),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(tf.nn.relu),
      tf.keras.layers.Dropout(rate=1-keep_prob), 

      tf.keras.layers.Dense(models[1]),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(tf.nn.relu),
      tf.keras.layers.Dropout(rate=1-keep_prob),    

      tf.keras.layers.Dense(models[2]),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(tf.nn.relu),
      tf.keras.layers.Dropout(rate=1-keep_prob), 

      tf.keras.layers.Dense(models[3]),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(tf.nn.relu),
      tf.keras.layers.Dropout(rate=1-keep_prob),

      tf.keras.layers.Dense(2, activation=tf.nn.softmax)
    ])      
  elif layer_num == 5:
      model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(models[0]),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(tf.nn.relu),
      tf.keras.layers.Dropout(rate=1-keep_prob), 

      tf.keras.layers.Dense(models[1]),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(tf.nn.relu),
      tf.keras.layers.Dropout(rate=1-keep_prob),    

      tf.keras.layers.Dense(models[2]),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(tf.nn.relu),
      tf.keras.layers.Dropout(rate=1-keep_prob), 

      tf.keras.layers.Dense(models[3]),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(tf.nn.relu),
      tf.keras.layers.Dropout(rate=1-keep_prob),

      tf.keras.layers.Dense(models[4]),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(tf.nn.relu),
      tf.keras.layers.Dropout(rate=1-keep_prob),

      tf.keras.layers.Dense(2, activation=tf.nn.softmax)
    ])             
  return model

In [0]:
n_split = 10
test_accurancy = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
train_accurancy = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [6]:
model_cnt = [7,8,9,10]

for i in model_cnt:
  
  x_train = np.loadtxt(file_path+'x_train'+str(i)+'(1).txt',dtype='float32')
  y_train = np.loadtxt(file_path+'y_train'+str(i)+'(1).txt',dtype='float32')
  x_test = np.loadtxt(file_path+'x_test'+str(i)+'(1).txt',dtype='float32')
  y_test = np.loadtxt(file_path+'y_test'+str(i)+'(1).txt',dtype='float32')

  models = [64,32,32,16,4]

  model_name = ''
  for p in range(len(models)):
    model_name = model_name + str(models[p]) + '_'
  keep_prob = 0.6 #0.6일때 성능이 더 좋았음.
  model_name = model_name + str(keep_prob) + '_' + str(i)
  model = create_model(models,keep_prob,len(models))
  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  model_json = model.to_json()
  with open(file_path+str(model_name)+".json", "w") as json_file: 
    json_file.write(model_json)

  print('##################################')
  print(str(i) + '-fold training model ')
  best_test_acc = 0
  best_train_acc = 0
  best_tmp = []
  best_y_test = []
  best_idx = 0
  with tf.device('/device:GPU:0'):
    for j in range(2000):
      model.fit(x=x_train, y=y_train, batch_size=256, epochs=200, verbose=0)
      tmp = np.asarray(model.predict(x_test))
      tmp = np.argmax(tmp, axis=1)
      print("%04d th train/test val:" % j)
      train_acc = model.evaluate(x_train, y_train)
      test_acc = model.evaluate(x_test, y_test)
      #테스트 정확도가 더 높을 때 업데이트
      if best_test_acc < test_acc[1] and j > 15:

        best_idx = j
        print(j)
        best_test_acc = test_acc[1]
        best_train_acc = train_acc[1]
        print('best_train_acc:', best_train_acc,
              ' best_test_acc:', best_test_acc)
        best_tmp = tmp
        best_y_test = y_test
        train_accurancy[i-1]=test_acc[1]
        test_accurancy[i-1]=test_acc[1]
        model.save_weights(file_path+str(model_name)+'_'+str(i) + '.h5')
        print('saved')

      #테스트 정확도가 똑같은데 트레인 정확도가 더 높다면 없데이트 
      elif best_test_acc !=0 and best_test_acc == test_acc[1] and best_train_acc < train_acc[1] :

        best_idx = j
        print(j)
        best_test_acc = test_acc[1]
        best_train_acc = train_acc[1]
        print('best_train_acc:', best_train_acc,
              ' best_test_acc:', best_test_acc)
        best_tmp = tmp
        best_y_test = y_test
        train_accurancy[i-1]=test_acc[1]
        test_accurancy[i-1]=test_acc[1]
        model.save_weights(file_path+str(model_name)+'_'+str(i) + '.h5')
        print('saved')

          
      
      print('number:', i ,'best_train_acc:', best_train_acc, ' best_test_acc:', best_test_acc)
      print(best_tmp)
      print(best_y_test)

  del model
  gc.collect()
  K.clear_session()




스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 1 1 1 1 1 1]
[1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1.
 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0.
 1. 0. 1. 1. 0. 1. 1. 1.]
1546 th train/test val:
80/80 [==============================] - 0s 64us/sample - loss: 4.3091 - acc: 0.6750
number: 10 best_train_acc: 0.9652778  best_test_acc: 0.8
[1 0 1 1 0 1 1 1 0 1 1 0 1 0 0 0 1 1 0 1 1 0 1 1 1 1 1 0 0 1 1 0 0 1 0 0 1
 0 1 1 0 0 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0
 1 1 1 1 1 1]
[1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1.
 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0.
 1. 0. 1. 1. 0. 1. 1. 1.]
1547 th train/test val:
80/80 [==============================] - 0s 77us/sample - loss: 4.1800 - acc: 0.6375
number: 10 best_train

In [0]:
def load_model(model_name):
  json_file = open(str(model_name),'r')
  loaded_model_json = json_file.read()

  json_file.close()
  model = tf.keras.models.model_from_json(loaded_model_json)
  return model

In [10]:
#'64_64_64_64_0.6_'
model_cnt = [1,2,3,4,5,6,7,8,9,10]
name = '64_32_32_16_4_0.6_'
for i in model_cnt:

  model_name = file_path + name + str(i) + '.json'
  loaded_model = load_model(model_name)

  loaded_model.build(input_shape=(256,22))
  model_weight = file_path + name + str(i) + '_' + str(i) + '.h5'
  loaded_model.load_weights(model_weight)
  print("Loaded model from disk")
  loaded_model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  x_train = np.loadtxt(file_path+'x_train'+str(i)+'(1).txt')
  y_train = np.loadtxt(file_path+'y_train'+str(i)+'(1).txt')
  x_test = np.loadtxt(file_path+'x_test'+str(i)+'(1).txt')
  y_test = np.loadtxt(file_path+'y_test'+str(i)+'(1).txt')

  test_acc = loaded_model.evaluate(x_test, y_test)
  train_acc = loaded_model.evaluate(x_train, y_train)
  test_accurancy[i-1] = test_acc[1]
  train_accurancy[i-1] = train_acc[1]
  print('test_acc :' + str(test_acc))
  tmp = np.asarray(loaded_model.predict(x_test))

Loaded model from disk
720/720 [==============================] - 0s 42us/sample - loss: 0.0184 - acc: 0.9972
test_acc :[1.3326440811157227, 0.775]
Loaded model from disk
720/720 [==============================] - 0s 49us/sample - loss: 0.0523 - acc: 0.9903
test_acc :[0.9873942017555237, 0.7625]
Loaded model from disk
720/720 [==============================] - 0s 42us/sample - loss: 0.0615 - acc: 0.9819
test_acc :[0.9830710649490356, 0.7125]
Loaded model from disk
720/720 [==============================] - 0s 44us/sample - loss: 0.0299 - acc: 0.9986
test_acc :[2.149533987045288, 0.725]
Loaded model from disk
720/720 [==============================] - 0s 53us/sample - loss: 0.0549 - acc: 0.9944
test_acc :[1.0679807662963867, 0.7375]
Loaded model from disk
720/720 [==============================] - 0s 44us/sample - loss: 0.0775 - acc: 0.9708
test_acc :[0.9334448218345642, 0.725]
Loaded model from disk
720/720 [==============================] - 0s 47us/sample - loss: 0.1196 - acc: 0.9778


In [11]:

test_accurancy=np.asarray(test_accurancy)
train_accurancy=np.asarray(train_accurancy)
test_average = np.sum(test_accurancy)/10
print('test average : ',test_average)
print('test acc : ',test_accurancy)
print('train acc : ', train_accurancy)


test average :  0.7625000476837158
test acc :  [0.775  0.7625 0.7125 0.725  0.7375 0.725  0.7875 0.7375 0.8625 0.8   ]
train acc :  [0.99722224 0.99027777 0.98194444 0.9986111  0.99444443 0.97083336
 0.9777778  0.99722224 1.         0.9652778 ]
